In [1]:
import sys
!{sys.executable} -m pip install rfpimp
!{sys.executable} -m pip install xgboost

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import learning_curve, cross_val_score, train_test_split, KFold

from rfpimp import *
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier

import matplotlib.pyplot as plt

In [3]:
# Load data
gene_expression = pd.read_csv('../Data/Processed/NSCLC_expression_model_training.csv', index_col='SampID')
clinical_data = pd.read_csv('../Data/Processed/NSCLC_labels_model_training.csv', index_col='SampID')

# Merge datasets
expression_clincov = pd.merge(gene_expression, clinical_data, left_index=True, right_index=True)
expression_condition = pd.merge(gene_expression, clinical_data[['Condition']], on='SampID', how='left')

print(f"Shape of merged data: {expression_condition.shape}")

Shape of merged data: (1109, 6142)


In [4]:
# Encode the "Condition" variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(expression_condition['Condition'])

# Prepare the feature matrix with gene expression only
X = expression_condition.drop(columns=['Condition'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Define Feature Selection Methods

In [5]:
def chi_squared_feature_selection(X, y, k=100):
    chi2_selector = SelectKBest(chi2, k=k)
    chi2_selector.fit(X, y)
    return X.columns[chi2_selector.get_support()].tolist()

def mutual_information_feature_selection(X, y, k=100):
    mi_selector = SelectKBest(mutual_info_classif, k=k)
    mi_selector.fit(X, y)
    return X.columns[mi_selector.get_support()].tolist()

def random_forest_feature_importance(X, y, k=100):
    model = RandomForestClassifier(random_state=42)
    model.fit(X, y)
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:k]
    return X.columns[indices].tolist()

def permutation_feature_importance(X, y, k=100):
    model = RandomForestClassifier(random_state=42)
    model.fit(X, y)
    result = permutation_importance(model, X, y, n_repeats=10, random_state=42)
    sorted_indices = result.importances_mean.argsort()[::-1][:k]
    return X.columns[sorted_indices].tolist()

def rfpimp_feature_selection(X, y_train, X_test, y_test, n_features=100):
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    imp = importances(rf, X_test, y_test, n_samples=-1)
    # Ensure imp is a DataFrame with 'Feature' as index and 'Importance' as a column
    if not isinstance(imp, pd.DataFrame) or 'Importance' not in imp.columns:
        raise ValueError("imp should be a DataFrame with 'Importance' column")
    return imp.nlargest(n_features, 'Importance').index.tolist()

### Function for training different feature sets using Random Forest Classifier and return metrics of performance

In [37]:
def train_and_evaluate(X_train, X_test, y_train, y_test, features):
    clf = RandomForestClassifier(random_state=42)
    
    # Cross-validation on training data
    cv_scores = cross_val_score(clf, X_train[features], y_train, cv=5)
    
    # Train on full training set
    clf.fit(X_train[features], y_train)
    
    # Predict on test set
    y_pred = clf.predict(X_test[features])
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return {
        'cv_score_mean': np.mean(cv_scores),
        'cv_score_std': np.std(cv_scores),
        'test_accuracy': accuracy,
        'test_precision': precision,
        'test_recall': recall,
        'test_f1': f1
    }

### Execution

In [38]:
# Main execution
feature_selection_methods = [
    ('Chi-Squared', chi_squared_feature_selection),
    ('Mutual Information', mutual_information_feature_selection),
    ('Random Forest', random_forest_feature_importance),
    ('Permutation', permutation_feature_importance),
    ('RFPIMP', rfpimp_feature_selection)
]

results = {}

for name, method in feature_selection_methods:
    if name == 'RFPIMP':
        selected_features = method(X_train, y_train, X_test, y_test)
    else:
        selected_features = method(X_train, y_train)
    results[name] = train_and_evaluate(X_train, X_test, y_train, y_test, selected_features)

# Display results
for method, metrics in results.items():
    print(f"\nResults for {method}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")


Results for Chi-Squared:
cv_score_mean: 0.9402
cv_score_std: 0.0181
test_accuracy: 0.9550
test_precision: 0.9573
test_recall: 0.9550
test_f1: 0.9548

Results for Mutual Information:
cv_score_mean: 0.9470
cv_score_std: 0.0137
test_accuracy: 0.9640
test_precision: 0.9645
test_recall: 0.9640
test_f1: 0.9639

Results for Random Forest:
cv_score_mean: 0.9515
cv_score_std: 0.0162
test_accuracy: 0.9685
test_precision: 0.9694
test_recall: 0.9685
test_f1: 0.9684

Results for Permutation:
cv_score_mean: 0.9414
cv_score_std: 0.0117
test_accuracy: 0.9505
test_precision: 0.9513
test_recall: 0.9505
test_f1: 0.9504

Results for RFPIMP:
cv_score_mean: 0.9425
cv_score_std: 0.0083
test_accuracy: 0.9505
test_precision: 0.9513
test_recall: 0.9505
test_f1: 0.9504


### Function for training different models using several feature sets and return metrics of performance

In [18]:
def train_and_evaluate_all_classifiers(X_train, X_test, y_train, y_test, features):

    classifiers = [
        ('Random Forest', RandomForestClassifier(random_state=42)),
        ('SVM', SVC(random_state=42)),
        ('XGBoost', XGBClassifier(random_state=42)),
        ('KNN', KNeighborsClassifier()),
        ('Naive Bayes', GaussianNB()),
        ('LDA', LinearDiscriminantAnalysis())
    ]
    
    results = {}
    
    for clf_name, clf in classifiers:
        # Cross-validation on training data
        cv_scores = cross_val_score(clf, X_train[features], y_train, cv=5)
        
        # Train on full training set
        clf.fit(X_train[features], y_train)
        
        # Predict on test set
        y_pred = clf.predict(X_test[features])
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        results[clf_name] = {
            'cv_score_mean': np.mean(cv_scores),
            'cv_score_std': np.std(cv_scores),
            'test_accuracy': accuracy,
            'test_precision': precision,
            'test_recall': recall,
            'test_f1': f1
        }
    
    return results

### Execution

In [19]:
# Main execution
def main_execution(X_train, y_train, X_test, y_test):
    
    feature_selection_methods = [
        ('Chi-Squared', chi_squared_feature_selection),
        ('Mutual Information', mutual_information_feature_selection),
        ('Random Forest', random_forest_feature_importance),
        ('Permutation', permutation_feature_importance),
        ('RFPIMP', rfpimp_feature_selection)
    ]

    all_results = {}

    for name, method in feature_selection_methods:
        if name == 'RFPIMP':
            selected_features = method(X_train, y_train, X_test, y_test)
        else:
            selected_features = method(X_train, y_train)
        all_results[name] = train_and_evaluate_all_classifiers(X_train, X_test, y_train, y_test, selected_features)

    # Display results
    for feature_method, classifiers_results in all_results.items():
        print(f"\nResults for {feature_method}:")
        for classifier, metrics in classifiers_results.items():
            print(f"  {classifier}:")
            for metric, value in metrics.items():
                print(f"    {metric}: {value:.4f}")

In [26]:
# Run the main execution
main_execution(X_train, y_train, X_test, y_test)


Results for Chi-Squared:
  Random Forest:
    cv_score_mean: 0.9402
    cv_score_std: 0.0181
    test_accuracy: 0.9550
    test_precision: 0.9573
    test_recall: 0.9550
    test_f1: 0.9548
  SVM:
    cv_score_mean: 0.9380
    cv_score_std: 0.0220
    test_accuracy: 0.9505
    test_precision: 0.9535
    test_recall: 0.9505
    test_f1: 0.9502
  XGBoost:
    cv_score_mean: 0.9470
    cv_score_std: 0.0127
    test_accuracy: 0.9459
    test_precision: 0.9465
    test_recall: 0.9459
    test_f1: 0.9459
  KNN:
    cv_score_mean: 0.9312
    cv_score_std: 0.0210
    test_accuracy: 0.9414
    test_precision: 0.9414
    test_recall: 0.9414
    test_f1: 0.9414
  Naive Bayes:
    cv_score_mean: 0.9278
    cv_score_std: 0.0153
    test_accuracy: 0.9459
    test_precision: 0.9482
    test_recall: 0.9459
    test_f1: 0.9457
  LDA:
    cv_score_mean: 0.9391
    cv_score_std: 0.0241
    test_accuracy: 0.9369
    test_precision: 0.9405
    test_recall: 0.9369
    test_f1: 0.9366

Results for Mutual In

### Explore feature sets with best performance and compare with feature set from DE analysis

In [6]:
random_forest_features = random_forest_feature_importance(X_train, y_train)
mutual_information_features = mutual_information_feature_selection(X_train, y_train)

In [9]:
biomarkers_from_de_analysis = pd.read_csv('../Data/Processed/NSCLC_biomarkers_de_analysis.csv')
de_features = biomarkers_from_de_analysis['x'].tolist()

In [10]:
# Convert all feature lists to sets for easier set operations
rf_set = set(random_forest_features)
mi_set = set(mutual_information_features)
de_set = set(de_features)

# Union of Random Forest and Mutual Information features
rf_mi_union = rf_set.union(mi_set)

# Overlap of Random Forest and Mutual Information features
rf_mi_overlap = rf_set.intersection(mi_set)

# Overlap of DE analysis and Random Forest features
de_rf_overlap = de_set.intersection(rf_set)

# Overlap of DE analysis and Mutual Information features
de_mi_overlap = de_set.intersection(mi_set)

# Print results
print(f"Number of Random Forest features: {len(rf_set)}")
print(f"Number of Mutual Information features: {len(mi_set)}")
print(f"Number of DE analysis features: {len(de_set)}")
print(f"\nNumber of features in union of RF and MI: {len(rf_mi_union)}")
print(f"Number of features in overlap of RF and MI: {len(rf_mi_overlap)}")
print(f"Number of features in overlap of DE and RF: {len(de_rf_overlap)}")
print(f"Number of features in overlap of DE and MI: {len(de_mi_overlap)}")

# If you want to see the actual features:
print("\nFeatures in overlap of RF and MI:")
print(rf_mi_overlap)
print("\nFeatures in overlap of DE and RF:")
print(de_rf_overlap)
print("\nFeatures in overlap of DE and MI:")
print(de_mi_overlap)

Number of Random Forest features: 100
Number of Mutual Information features: 100
Number of DE analysis features: 100

Number of features in union of RF and MI: 151
Number of features in overlap of RF and MI: 49
Number of features in overlap of DE and RF: 8
Number of features in overlap of DE and MI: 9

Features in overlap of RF and MI:
{'ENSG00000272502.1', 'ENSG00000224090.1', 'ENSG00000086570.12', 'ENSG00000188959.9', 'ENSG00000250343.1', 'ENSG00000189280.3', 'ENSG00000185499.16', 'ENSG00000250522.1', 'ENSG00000260581.1', 'ENSG00000234352.9', 'ENSG00000271134.1', 'ENSG00000200033.1', 'ENSG00000157873.17', 'ENSG00000114270.17', 'ENSG00000279141.3', 'ENSG00000163961.4', 'ENSG00000226015.2', 'ENSG00000253746.1', 'ENSG00000236740.6', 'ENSG00000161203.13', 'ENSG00000124406.16', 'ENSG00000253706.5', 'ENSG00000254343.2', 'ENSG00000196344.11', 'ENSG00000065809.13', 'ENSG00000204544.5', 'ENSG00000169474.4', 'ENSG00000248554.1', 'ENSG00000069812.11', 'ENSG00000234896.1', 'ENSG00000219375.1', '

In [11]:
# Overlap of DE analysis, Mutual Information and Random Forest features
de_rf_mi_overlap = de_rf_overlap.intersection(de_mi_overlap)

# Union of DE analysis, Mutual Information and Random Forest features
de_rf_mi_union = de_rf_overlap.union(de_mi_overlap)

print(f"Number of features in overlap of DE, RF and MI: {len(de_rf_mi_overlap)}")
print(f"Number of features in union of DE, RF and MI: {len(de_rf_mi_union)}")

Number of features in overlap of DE, RF and MI: 0
Number of features in union of DE, RF and MI: 17


### Experiment with only top 50 features

In [8]:
random_forest_features_50 = random_forest_feature_importance(X_train, y_train, k=50)
mutual_information_features_50 = mutual_information_feature_selection(X_train, y_train, k=50)

In [13]:
biomarkers_50_from_de_analysis = pd.read_csv('../Data/Processed/NSCLC_biomarkers_50_de_analysis.csv')
de_50_features = biomarkers_50_from_de_analysis['x'].tolist()

In [17]:
# Convert all feature lists to sets for easier set operations
rf_set_50 = set(random_forest_features_50)
mi_set_50 = set(mutual_information_features_50)
de_set_50 = set(de_50_features)

# Union of Random Forest and Mutual Information features
rf_mi_union_50 = rf_set_50.union(mi_set_50)

# Overlap of Random Forest and Mutual Information features
rf_mi_overlap_50 = rf_set_50.intersection(mi_set_50)

# Overlap of DE analysis and Random Forest features
de_rf_overlap_50 = de_set.intersection(rf_set_50)

# Overlap of DE analysis and Mutual Information features
de_mi_overlap_50 = de_set.intersection(mi_set_50)

# Print results
print(f"Number of Random Forest features: {len(rf_set_50)}")
print(f"Number of Mutual Information features: {len(mi_set_50)}")
print(f"Number of DE analysis features: {len(de_set_50)}")
print(f"\nNumber of features in union of RF and MI: {len(rf_mi_union_50)}")
print(f"Number of features in overlap of RF and MI: {len(rf_mi_overlap_50)}")
print(f"Number of features in overlap of DE and RF: {len(de_rf_overlap_50)}")
print(f"Number of features in overlap of DE and MI: {len(de_mi_overlap_50)}")

# If you want to see the actual features:
print("\nFeatures in overlap of RF and MI:")
print(rf_mi_overlap_50)
print("\nFeatures in overlap of DE and RF:")
print(de_rf_overlap_50)
print("\nFeatures in overlap of DE and MI:")
print(de_mi_overlap_50)

Number of Random Forest features: 50
Number of Mutual Information features: 50
Number of DE analysis features: 50

Number of features in union of RF and MI: 78
Number of features in overlap of RF and MI: 22
Number of features in overlap of DE and RF: 1
Number of features in overlap of DE and MI: 4

Features in overlap of RF and MI:
{'ENSG00000279141.3', 'ENSG00000165215.6', 'ENSG00000271134.1', 'ENSG00000185499.16', 'ENSG00000234352.9', 'ENSG00000249894.1', 'ENSG00000219375.1', 'ENSG00000272502.1', 'ENSG00000213495.3', 'ENSG00000086570.12', 'ENSG00000114270.17', 'ENSG00000248554.1', 'ENSG00000161203.13', 'ENSG00000224984.1', 'ENSG00000180438.15', 'ENSG00000249746.1', 'ENSG00000226015.2', 'ENSG00000250343.1', 'ENSG00000260581.1', 'ENSG00000204544.5', 'ENSG00000196344.11', 'ENSG00000189280.3'}

Features in overlap of DE and RF:
{'ENSG00000225062.1'}

Features in overlap of DE and MI:
{'ENSG00000204253.4', 'ENSG00000226084.5', 'ENSG00000106125.14', 'ENSG00000256968.1'}


In [31]:
def plot_learning_curve(estimator, X, y, title):
    """
    Plot learning curve for a given estimator.
    
    Args:
    estimator: The classifier
    X: Features
    y: Target variable
    title: Title of the plot
    """
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, n_jobs=-1, 
        train_sizes=np.linspace(0.1, 1.0, 5))
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    plt.legend(loc="best")
    return plt

In [32]:
def nested_cross_validation(X, y, classifier, feature_set, cv_outer=5, cv_inner=3):
    """
    Perform nested cross-validation.
    
    Args:
    X: Features (pandas DataFrame)
    y: Target variable (numpy array)
    classifier: The classifier to use
    feature_set: List of features to use
    cv_outer: Number of folds for outer CV
    cv_inner: Number of folds for inner CV
    
    Returns:
    dict: Results of nested cross-validation
    """
    cv_outer = KFold(n_splits=cv_outer, shuffle=True, random_state=42)
    cv_inner = KFold(n_splits=cv_inner, shuffle=True, random_state=42)

    nested_scores = {
        'accuracy': [], 'precision': [], 'recall': [], 'f1': []
    }

    for train_ix, test_ix in cv_outer.split(X):
        # Use integer-based indexing for both X and y
        X_train, X_test = X.iloc[train_ix][feature_set], X.iloc[test_ix][feature_set]
        y_train, y_test = y[train_ix], y[test_ix]

        # Inner cross-validation for hyperparameter tuning (if needed)
        # For simplicity, we're not doing hyperparameter tuning here
        
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        
        nested_scores['accuracy'].append(accuracy_score(y_test, y_pred))
        nested_scores['precision'].append(precision_score(y_test, y_pred, average='weighted'))
        nested_scores['recall'].append(recall_score(y_test, y_pred, average='weighted'))
        nested_scores['f1'].append(f1_score(y_test, y_pred, average='weighted'))

    results = {}
    for metric in nested_scores.keys():
        results[f'{metric}_mean'] = np.mean(nested_scores[metric])
        results[f'{metric}_std'] = np.std(nested_scores[metric])

    return results

In [33]:
def evaluate_feature_sets(X, y, feature_sets, classifiers):
    """
    Evaluate different feature sets using various classifiers.
    
    Args:
    X: Features (pandas DataFrame)
    y: Target variable (numpy array)
    feature_sets: Dictionary of feature sets to evaluate
    classifiers: Dictionary of classifiers to use
    
    Returns:
    dict: Results of evaluation for each feature set and classifier combination
    """
    results = {}

    for set_name, features in feature_sets.items():
        # Ensure we're only using features that exist in X
        valid_features = [f for f in features if f in X.columns]
        X_selected = X[valid_features]
        
        for clf_name, clf in classifiers.items():
            # Generate and save learning curve plot
            plt = plot_learning_curve(clf, X_selected, y, f'Learning Curve - {set_name} - {clf_name}')
            plt.savefig(f'learning_curve_{set_name}_{clf_name}.png')
            plt.close()

            # Perform nested cross-validation
            nested_results = nested_cross_validation(X, y, clf, valid_features)
            
            # Store results
            results[f'{set_name}_{clf_name}'] = nested_results

    return results

In [34]:
def main_execution(X, y):
    """
    Main execution function to run the entire evaluation process.
    
    Args:
    X: Features (pandas DataFrame)
    y: Target variable (numpy array)
    """
    # Define feature sets
    feature_sets = {
        'Random Forest': random_forest_features,
        'Mutual Information': mutual_information_features,
        'DE Analysis': de_features,
        'RF_MI_Intersection': list(set(random_forest_features) & set(mutual_information_features)),
        'RF_MI_Union': list(set(random_forest_features) | set(mutual_information_features)),
        'DE_RF_Intersection': list(set(random_forest_features) & set(de_features)),
        'DE_MI_Intersection': list(set(mutual_information_features) & set(de_features)),
        'RF_MI_DE_Union': list(set(de_rf_overlap) | set(de_mi_overlap))
    }

    # Ensure all features in feature sets exist in X
    for set_name, features in feature_sets.items():
        valid_features = [f for f in features if f in X.columns]
        feature_sets[set_name] = valid_features
        print(f"Feature set '{set_name}' has {len(valid_features)} valid features out of {len(features)} original features.")

    # Define classifiers
    classifiers = {
        'Random Forest': RandomForestClassifier(random_state=42),
        'SVM': SVC(random_state=42),
        'XGBoost': XGBClassifier(random_state=42),
        'KNN': KNeighborsClassifier(),
        'Naive Bayes': GaussianNB(),
        'LDA': LinearDiscriminantAnalysis()
    }

    # Evaluate all feature sets with all classifiers
    results = evaluate_feature_sets(X, y, feature_sets, classifiers)

    # Display results
    for key, value in results.items():
        print(f"\nResults for {key}:")
        for metric, score in value.items():
            print(f"  {metric}: {score:.4f}")
        
    # Important metrics to focus on:
    # 1. accuracy_mean: Overall performance of the model
    # 2. f1_mean: Balanced measure of precision and recall
    # 3. accuracy_std and f1_std: Consistency of the model's performance
    
    # Additional analysis: Compare number of features vs performance
    for set_name, features in feature_sets.items():
        print(f"\nFeature set: {set_name}")
        print(f"Number of features: {len(features)}")

In [35]:
# Run the main execution
main_execution(X_train, y_train)

Feature set 'Random Forest' has 100 valid features out of 100 original features.
Feature set 'Mutual Information' has 100 valid features out of 100 original features.
Feature set 'DE Analysis' has 100 valid features out of 100 original features.
Feature set 'RF_MI_Intersection' has 50 valid features out of 50 original features.
Feature set 'RF_MI_Union' has 150 valid features out of 150 original features.
Feature set 'DE_RF_Intersection' has 8 valid features out of 8 original features.
Feature set 'DE_MI_Intersection' has 10 valid features out of 10 original features.
Feature set 'RF_MI_DE_Union' has 18 valid features out of 18 original features.

Results for Random Forest_Random Forest:
  accuracy_mean: 0.9492
  accuracy_std: 0.0235
  precision_mean: 0.9504
  precision_std: 0.0228
  recall_mean: 0.9492
  recall_std: 0.0235
  f1_mean: 0.9491
  f1_std: 0.0236

Results for Random Forest_SVM:
  accuracy_mean: 0.9470
  accuracy_std: 0.0213
  precision_mean: 0.9483
  precision_std: 0.0204
 

In [12]:
# Define each list for saving
feature_lists = {
    'Random_Forest': random_forest_features,
    'RF_MI_Intersection': list(set(random_forest_features) & set(mutual_information_features)),
    'RF_MI_Union': list(set(random_forest_features) | set(mutual_information_features)),
    'RF_MI_DE_Union': list(set(de_rf_overlap) | set(de_mi_overlap))
}

# Save each list to a CSV file
for list_name, feature_list in feature_lists.items():
    # Convert list to DataFrame
    df = pd.DataFrame(feature_list, columns=['x'])  
    # Save DataFrame as CSV
    df.to_csv(f'{list_name}.csv', index=False)            